# Day 07: Amplification Circuit
## Part One

In [173]:
from collections import deque
class Intcode:
    def __init__(self, program, inp=[]):
        self.iter = None
        self.codes = [int(i) for i in program.split(',')]
        self.inp = deque(inp)
        self.base = 0
        self.pos = 0
        self.ops = {
            1: self._add,
            2: self._multiply,
            #3: self._inpt,
            #4: self._outpt,
            5: self._jump_if_true,
            6: self._jump_if_false,
            7: self._less_than,
            8: self._equals,
            9: self._adjust_base,
            99: self._halt
        }
        
    def run(self):
        if self.iter is None:
            self.iter = self._run()
        return next(self.iter)
    
    def _run(self):
        while self.pos < len(self.codes):
            opcode = self.codes[self.pos] % 100

            if opcode == 4:
                self.pos, out = self._outpt()
                yield out
            elif opcode == 3:
                self.pos = self._inpt()
            else:
                self.pos = self.ops[opcode]()
                
    def add_input(self, *args):
        self.inp.extend(args)
        return self
                
    def _get_params(self, num):
        res = []
        modes = list(str(self.codes[self.pos])[:-2])
        for arg in self.codes[self.pos+1:self.pos+num+1]:
            if len(modes) > 0:
                m = int(modes.pop())
            else:
                m = 0
            if m == 0:
                res.append(self._r(arg))
            elif m == 1:
                res.append(arg)
            elif m == 2:
                res.append(self._r(self.base + arg))
        return res if len(res) > 1 else res[0]

    def _get_out_pos(self, pos, offset):
        modes = list(str(self.codes[pos])[:-2])
        if len(modes) >= offset and modes[0] == "2":
            return self.base + self._r(pos+offset)
        return self.codes[pos + offset]
    
    def _expand(self, pos):
        if pos >= len(self.codes):
            self.codes.extend([0]*(pos-len(self.codes) + 1))

    def _r(self, pos):
        self._expand(pos)
        return self.codes[pos]

    def _w(self, pos, offset, val):
        write_pos = self._get_out_pos(pos, offset)
        self._expand(write_pos)
        self.codes[write_pos] = val
        
    def _outpt(self):
        a = self._get_params(1)
        return self.pos + 2, a
    
    def _inpt(self):
        self._w(self.pos, 1, self.inp.popleft())
        return self.pos + 2
    
    def _add(self):
        a,b = self._get_params(2)
        self._w(self.pos, 3, a + b)
        return self.pos + 4
    
    def _multiply(self):
        a,b = self._get_params(2)
        self._w(self.pos, 3, a * b)
        return self.pos + 4
    
    def _halt(self):
        return float("inf")

    def _jump_if_true(self):
        a,b = self._get_params(2)
        if a != 0:
            return b
        return self.pos+3

    def _jump_if_false(self):
        a,b = self._get_params(2)
        if a == 0:
            return b
        return self.pos+3

    def _less_than(self):
        a,b = self._get_params(2)
        self._w(self.pos, 3, int(a < b))
        return self.pos+4

    def _equals(self):
        a,b = self._get_params(2)
        self._w(self.pos, 3, int(a==b))
        return self.pos+4

    def _adjust_base(self):
        a = self._get_params(1)
        self.base += a
        return self.pos + 2

In [4]:
import itertools

In [175]:
with open('input/7_1.txt') as f:
    program = f.read().strip()

In [176]:
def amplify(program, phases):
    out = 0
    for phase in phases:
        out = Intcode(program, [phase, out]).run()
    return out

In [177]:
max(amplify(program, perm) for perm in (itertools.permutations(range(5),5)))

118936

## Part Two

In [143]:
program = "3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10"
phases = [9,7,8,5,6]

In [180]:
def amplify(program, phases):
    amps = [Intcode(program).add_input(phase) for phase in phases]

    out = 0
    for amp in itertools.cycle(amps):
        try:
            out = amp.add_input(out).run()
        except StopIteration:
            return out

In [5]:
with open('input/7_1.txt') as f:
    program = f.read().strip()
max(amplify(program, perm) for perm in (itertools.permutations(range(5,10),5)))

57660948